In [73]:
import os
import json
import pandas as pd

In [2]:
def get_keys(path):
    with open(path) as f:
        return json.load(f)

In [3]:
pwd

'/workspaces/data-engineering-capstone/notebook'

In [9]:
uri = "../temp/matches.json"

In [7]:
os.path.exists(uri)

True

In [12]:
with open(uri) as file:
    data = json.load(file)

In [65]:
# keys = data['Premier League'][0].keys()
# keys

In [26]:
pl_data = data['Premier League'][0]

In [68]:
# To get data of second matchday
# data['Premier League'][1]

In [30]:
season = int(pl_data['filters']['season'])
season

2023

In [33]:
matchday = int(pl_data['filters']['matchday'])
matchday

1

In [66]:
# pl_data['resultSet']

##### FIRST_MATCHDAY = pl_data['resultSet']['first']
##### LAST_MATCHDAY = pl_data['resultSet']['last']

##### FIRST_MATCHDAY, LAST_MATCHDAY

In [39]:
LEAGUE_ID = pl_data['competition']['id']

LEAGUE_ID

2021

In [63]:
# pl_data['matches']

In [62]:
# pl_data['matches'][0]

In [64]:
# pl_data['matches'][0].keys()

In [46]:
FIRST_MATCHDAY = pl_data['matches'][0]['season']['startDate']
LAST_MATCHDAY = pl_data['matches'][0]['season']['endDate']
FIRST_MATCHDAY, LAST_MATCHDAY

('2023-08-11', '2024-05-19')

In [47]:
match_id = pl_data['matches'][0]['id']

match_id

435943

In [49]:
matchdate = pl_data['matches'][0]['utcDate']
matchdate

'2023-08-11T19:00:00Z'

In [54]:
home_team = pl_data['matches'][0]['homeTeam']['shortName']
home_team

'Burnley'

In [55]:
away_team = pl_data['matches'][0]['awayTeam']['shortName']
away_team

'Man City'

In [56]:
home_score = pl_data['matches'][0]['score']['fullTime']['home']
home_score

0

In [57]:
away_score = pl_data['matches'][0]['score']['fullTime']['away']
away_score

3

In [86]:
def get_match_points(home_score: int, away_score: int) -> tuple[int, int]:

    if home_score == away_score:
        home_team_points, away_team_points = 1, 1
    elif home_score > away_score:
        home_team_points = 3
        away_team_points = 0
    else:
        home_team_points = 0
        away_team_points = 3
    
    return home_team_points, away_team_points 

In [87]:
def get_goal_difference(home_score: int, away_score: int) -> tuple[int, int]:
    home_gd = home_score - away_score

    return home_gd, -home_gd

In [61]:
match_referee = pl_data['matches'][0]['referees'][0]['name']
match_referee

'Craig Pawson'

In [95]:
leagues = {'Premier League': [38, 10], 
          'Bundesliga': [34, 9], 
          'Ligue 1': [38, 10], 
          'La Liga': [38, 10], 
          'Serie A': [38, 10]}

In [94]:
columns = ['league', 'season', 'league_id', 'season_start_date', 'season_end_date', 
           'matchday', 'match_id', 'match_date', 'home_team', 'away_team', 
           'home_score', 'away_score', 'home_team_points_earned', 'away_team_points_earned', 
           'home_team_gd', 'away_team_gd', 'match_referee']

df = pd.DataFrame(columns = columns)

In [98]:
for league, total_matchdays in leagues.items():

    # Get Season, League_id, first and last matchday dates only once as that is a constant
    firstday_data = data[league][0]
    SEASON = int(firstday_data['filters']['season'])
    LEAGUE_ID = firstday_data['competition']['id']
    FIRST_MATCHDAY = firstday_data['matches'][0]['season']['startDate']
    LAST_MATCHDAY = firstday_data['matches'][0]['season']['endDate']
    
    # Also need to consider status
    for mday in range(0, total_matchdays[0]):
        pl_data = data[league][mday]
        
        matchday = int(pl_data['filters']['matchday'])
    
        for match_num in range(0, total_matchdays[1]):
            match_data = pl_data['matches'][match_num]
            match_id = match_data['id']
            matchdate = match_data['utcDate']
            home_team = match_data['homeTeam']['shortName']
            away_team = match_data['awayTeam']['shortName']
            home_score = match_data['score']['fullTime']['home']
            away_score = match_data['score']['fullTime']['away']
                  
            try:
                match_referee = match_data['referees'][0]['name']
            except IndexError:
                match_referee = ''
            
            home_team_points, away_team_points = get_match_points(home_score=home_score, away_score=away_score)
            
            try:
                home_team_gd, away_team_gd = get_goal_difference(home_score=home_score, away_score=away_score)
            except TypeError:
                home_team_gd, away_team_gd = None, None
                
            df.loc[len(df.index)] = [league, SEASON, LEAGUE_ID, FIRST_MATCHDAY, LAST_MATCHDAY, 
                                     matchday, match_id, matchdate, home_team, away_team, 
                                     home_score, away_score, home_team_points, away_team_points, 
                                     home_team_gd, away_team_gd, match_referee]

IndexError: list index out of range

In [79]:
pl_data['matches'][10]['referees']

IndexError: list index out of range

In [90]:
df.head(10).T

,0,1,2,3,4,5,6,7,8,9
season,2023,2023,2023,2023,2023,2023,2023,2023,2023,2023
league_id,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021
season_start_date,2023-08-11,2023-08-11,2023-08-11,2023-08-11,2023-08-11,2023-08-11,2023-08-11,2023-08-11,2023-08-11,2023-08-11
season_end_date,2024-05-19,2024-05-19,2024-05-19,2024-05-19,2024-05-19,2024-05-19,2024-05-19,2024-05-19,2024-05-19,2024-05-19
matchday,1,1,1,1,1,1,1,1,1,1
match_id,435943,435944,435945,435946,435947,435948,435949,435950,435951,435952
match_date,2023-08-11T19:00:00Z,2023-08-12T12:00:00Z,2023-08-12T14:00:00Z,2023-08-12T14:00:00Z,2023-08-12T14:00:00Z,2023-08-12T14:00:00Z,2023-08-12T16:30:00Z,2023-08-13T13:00:00Z,2023-08-13T15:30:00Z,2023-08-14T19:00:00Z
home_team,Burnley,Arsenal,Bournemouth,Brighton Hove,Everton,Sheffield Utd,Newcastle,Brentford,Chelsea,Man United
away_team,Man City,Nottingham,West Ham,Luton Town,Fulham,Crystal Palace,Aston Villa,Tottenham,Liverpool,Wolverhampton
home_score,0,2,1,4,0,0,5,2,1,1
